In [ ]:
# Instalar dependencias necesarias
%pip install -q flask pyngrok transformers scipy

# Crear la estructura del proyecto
import os

os.makedirs("app", exist_ok=True)
os.makedirs("output", exist_ok=True)

# Crear el archivo del servidor Flask (app/server.py)
server_code = """
from flask import Flask, render_template_string, send_file, request
from transformers import pipeline
import scipy
import time
import os

app = Flask(__name__)

HTML_PAGE = '''
<!DOCTYPE html>
<html>
<head>
    <title>Generador de Melodía</title>
</head>
<body style="font-family: Arial; text-align: center; margin-top: 100px;">
    <h1>🎵 Generador de Música Algorítmica</h1>
    <form method="post" action="/generate">
        <button type="submit" style="padding: 10px 20px; font-size: 16px;">Generar Melodía</button>
    </form>
    {% if url %}
        <p><a href="{{ url }}" download>⬇️ Descargar audio generado</a></p>
    {% endif %}
</body>
</html>
'''

@app.route('/', methods=["GET"])
def index():
    return render_template_string(HTML_PAGE)

@app.route('/generate', methods=["POST"])
def generate():
    output_path = "output/musicgen_out.wav"
    start_time = time.time()
    print("Cargando modelo...")
    synthesiser = pipeline("text-to-audio", "facebook/musicgen-small")
    print("Modelo cargado. Generando música...")
    music = synthesiser("", forward_params={"do_sample": True})
    scipy.io.wavfile.write(output_path, rate=music["sampling_rate"], data=music["audio"])
    print("Audio generado en %.2f segundos" % (time.time() - start_time))
    return render_template_string(HTML_PAGE, url=f"/download")

@app.route('/download')
def download():
    path = "output/musicgen_out.wav"
    if os.path.exists(path):
        return send_file(path, as_attachment=True)
    return "Archivo no encontrado", 404

if __name__ == '__main__':
    app.run()
"""
with open("app/server.py", "w") as f:
    f.write(server_code)

# Iniciar el servidor Flask con pyngrok
from pyngrok import ngrok
import threading
import time

# Abrir el túnel
public_url = ngrok.connect(5000)
print("Tu app está disponible en:", public_url)

# Ejecutar el servidor en otro hilo
def run_flask():
    os.system("python3 app/server.py")

thread = threading.Thread(target=run_flask)
thread.start()


: 